In [ ]:
  # Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install numpy==1.23.5 mediapipe==0.10.21

In [ ]:
import mediapipe as mp

In [ ]:
!find /content/kata_elements -type d -exec sh -c 'echo -n "{}: "; find "{}" -maxdepth 1 -type f | wc -l' \;

/content/kata_elements: 0
/content/kata_elements/BLOCKS: 0
/content/kata_elements/BLOCKS/r-jodan-haiwan-uke: 210
/content/kata_elements/BLOCKS/l-chudan-shuto-uke: 842
/content/kata_elements/BLOCKS/r-jodan-shuto-age-uke: 210
/content/kata_elements/BLOCKS/l-jodan-age-uke: 353
/content/kata_elements/BLOCKS/r-chudan-shuto-uke: 580
/content/kata_elements/BLOCKS/l-chudan-gyaku-uchi-uke: 373
/content/kata_elements/BLOCKS/l-jodan-haiwan-uke: 206
/content/kata_elements/BLOCKS/r-gedan-barai: 204
/content/kata_elements/BLOCKS/r-morote-uke: 278
/content/kata_elements/BLOCKS/r-chudan-gyaku-uchi-uke: 388
/content/kata_elements/BLOCKS/l-gedan-barai: 204
/content/kata_elements/BLOCKS/l-jodan-shuto-age-uke: 215
/content/kata_elements/BLOCKS/r-jodan-age-uke: 180
/content/kata_elements/BLOCKS/l-chudan-osae-uke: 400
/content/kata_elements/KICKS: 0
/content/kata_elements/KICKS/l-chudan-mae-geri: 179
/content/kata_elements/KICKS/r-jodan-yoko-keage: 188
/content/kata_elements/KICKS/r-chudan-mae-geri: 184
/co

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import logging

# Настройка логирования
logger = logging.getLogger()  # Получаем корневой логгер
logger.setLevel(logging.INFO)  # Устанавливаем уровень логирования

# Очищаем все существующие обработчики
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Добавляем только FileHandler
file_handler = logging.FileHandler("processing.log")
file_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(file_handler)

In [ ]:
# Отключаем логирование сторонних библиотек
logging.getLogger("yolo").setLevel(logging.WARNING)  # Пример для YOLO
logging.getLogger("mediapipe").setLevel(logging.WARNING)  # Пример для MediaPipe

In [ ]:
#@title Оцениваем изображения с tqdm
import os
import cv2
import json
from tqdm import tqdm
import logging
import mediapipe as mp
from ultralytics import YOLO

# Инициализация MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# Загрузка предобученной модели YOLO
yolo_model = YOLO('yolov8n.pt')  # Используем маленькую версию YOLOv8

# Путь к папке с изображениями
base_dir = '/content/kata_elements'

# Создаем папку для сохранения JSON файлов
output_dir = '/content/kata_elements_output'
os.makedirs(output_dir, exist_ok=True)

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Уровень логирования
    format="%(asctime)s - %(levelname)s - %(message)s",  # Формат сообщений
    handlers=[
        logging.FileHandler("processing.log")  # Сохранение логов только в файл
    ]
)

# Словарь для хранения статистики по каждому классу
class_stats = {}

# Рекурсивно обходим все подпапки
for root, dirs, files in os.walk(base_dir):
    if files:  # Обрабатываем только папки с изображениями
        class_name = os.path.basename(root)  # Название класса (название папки)
        image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not image_files:
            continue  # Пропускаем пустые папки

        logging.info(f"Обработка класса: {class_name}")

        # Список для хранения координат всех поз
        all_poses_data = []

        # Счетчик распознанных изображений для текущего класса
        recognized_count = 0

        # Обрабатываем каждое изображение с tqdm
        for image_file in tqdm(image_files, desc=f"Processing {class_name}", position=0, leave=True):
            # Полный путь к изображению
            image_path = os.path.join(root, image_file)

            # Загрузка изображения
            image = cv2.imread(image_path)
            if image is None:
                logging.warning(f"Не удалось загрузить изображение: {image_path}")
                continue

            # Конвертация изображения в RGB (YOLO работает с RGB)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Обнаружение людей с помощью YOLO
            results = yolo_model(image_rgb, verbose=False)

            # Ищем bounding box человека (класс "person" имеет ID 0 в YOLO)
            person_detected = False
            for result in results:
                boxes = result.boxes  # Bounding boxes
                for box in boxes:
                    if int(box.cls) == 0:  # Класс "person"
                        person_detected = True

                        # Получаем координаты bounding box
                        x1, y1, x2, y2 = map(int, box.xyxy[0])

                        # Обрезаем изображение по bounding box
                        cropped_image = image[y1:y2, x1:x2]

                        # Конвертация обрезанного изображения в RGB для MediaPipe
                        cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

                        # Распознавание позы на обрезанном изображении
                        pose_results = pose.process(cropped_image_rgb)

                        # Если найдена поза
                        if pose_results.pose_landmarks:
                            # Проверяем visibility для точек с индексами 11 по 32
                            landmarks = pose_results.pose_landmarks.landmark
                            valid_pose = all(
                                landmarks[i].visibility > 0.1 for i in range(11, 33)
                            )

                            if valid_pose:
                                # Увеличиваем счетчик распознанных изображений
                                recognized_count += 1

                                # Сохраняем координаты
                                pose_data = {
                                    "image": image_file,
                                    "landmarks": [
                                        {
                                            "id": i,
                                            "x": landmark.x,
                                            "y": landmark.y,
                                            "z": landmark.z,
                                            "visibility": landmark.visibility
                                        }
                                        for i, landmark in enumerate(pose_results.pose_landmarks.landmark)
                                    ]
                                }
                                all_poses_data.append(pose_data)
                            else:
                                logging.warning(f"На изображении {image_file} не все ключевые точки видны (visibility > 0.1).")
                        else:
                            logging.warning(f"На изображении {image_file} поза не найдена.")

            if not person_detected:
                logging.warning(f"На изображении {image_file} человек не обнаружен.")

        # Сохраняем все координаты в JSON для текущего класса
        json_path = os.path.join(output_dir, f'{class_name}_poses_data.json')
        if all_poses_data:  # Сохраняем JSON только если есть данные
            with open(json_path, 'w') as f:
                json.dump(all_poses_data, f, indent=4)
            logging.info(f"Координаты сохранены в {json_path}")
        else:
            logging.warning(f"Нет данных для сохранения в JSON для класса {class_name}.")

        # Логируем количество распознанных изображений
        total_images = len(image_files)
        logging.info(f"Класс: {class_name}, всего изображений: {total_images}, распознано: {recognized_count}")

        # Сохраняем статистику для текущего класса
        class_stats[class_name] = {
            "total_images": total_images,
            "recognized_images": recognized_count
        }

# Сохраняем общую статистику в файл
stats_path = os.path.join(output_dir, "class_statistics.json")
with open(stats_path, 'w') as f:
    json.dump(class_stats, f, indent=4)
logging.info(f"Статистика по классам сохранена в {stats_path}")

Processing r-chudan-yoko-tettsui: 100%|██████████| 185/185 [01:03<00:00,  2.89it/s]


In [ ]:
/content/drive/MyDrive/Karate2/dataset

In [ ]:
import os
import json

# Путь к папке с JSON-файлами
folder_path = '/content/drive/MyDrive/Karate2/dataset_01_full'

# Функция для подсчета количества изображений в JSON-файле
def count_images_in_json(data):
    """
    Подсчитывает количество записей с полем "image" в JSON-файле.
    """
    if isinstance(data, list):  # Проверяем, что данные — это список
        return len([item for item in data if "image" in item])
    return 0

# Словарь для хранения результатов
file_image_counts = {}

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):  # Обрабатываем только JSON-файлы
        file_path = os.path.join(folder_path, filename)

        try:
            # Читаем содержимое JSON-файла
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)

            # Подсчитываем количество изображений
            image_count = count_images_in_json(data)
            file_image_counts[filename] = image_count
        except Exception as e:
            print(f"Ошибка при обработке файла {filename}: {e}")

# Вывод результатов
print("Количество изображений в каждом файле:")
for filename, count in file_image_counts.items():
    print(f"{filename}: {count} изображений")

# Общая статистика
total_files = len(file_image_counts)
total_images = sum(file_image_counts.values())
print(f"\nОбщее количество файлов: {total_files}")
print(f"Общее количество изображений: {total_images}")

Количество изображений в каждом файле:
l-turn-270_poses_data.json: 188 изображений
l-chudan-osae-uke_poses_data.json: 172 изображений
r-zenkutsu-dachi-hanmi_poses_data.json: 162 изображений
r-chudan-gyaku-uchi-uke_poses_data.json: 210 изображений
l-chudan-shuto-uke_poses_data.json: 399 изображений
r-morote-uke_poses_data.json: 127 изображений
r-chudan-shuto-uke_poses_data.json: 266 изображений
r-chudan-yoko-tettsui_poses_data.json: 98 изображений
r-gedan-barai_poses_data.json: 81 изображений
r-tsuru-ashi-dachi_poses_data.json: 90 изображений
l-chudan-mae-geri_poses_data.json: 77 изображений
r-jodan-shuto-age-uke_poses_data.json: 101 изображений
l-zenkutsu-dachi_poses_data.json: 326 изображений
rei_poses_data.json: 444 изображений
l-kokutsu-dachi_poses_data.json: 533 изображений
l-zenkutsu-dachi-hanmi_poses_data.json: 214 изображений
l-turn-45_poses_data.json: 191 изображений
r-jodan-haiwan-uke_poses_data.json: 106 изображений
class_statistics.json: 0 изображений
l-turn-90_poses_data.js

In [ ]:
#Oversampling (увеличение числа примеров для малых классов):
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

In [ ]:
#@title Обучение с весами
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

# Путь к папке с JSON-файлами
folder_path = '/content/drive/MyDrive/Karate2/dataset_01_full'

# Функция для извлечения данных из JSON-файла
def extract_features_and_labels(folder_path):
    features = []
    labels = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            class_name = filename.split("_")[0]  # Извлекаем имя класса из имени файла

            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)

            for item in data:
                if "landmarks" in item and len(item["landmarks"]) > 0:
                    # Извлекаем координаты landmarks
                    landmarks = item["landmarks"]
                    coords = [coord for landmark in landmarks for coord in [landmark["x"], landmark["y"], landmark["z"]]]
                    features.append(coords)
                    labels.append(class_name)

    return np.array(features), np.array(labels)

# Извлечение данных
features, labels = extract_features_and_labels(folder_path)

# Преобразование меток классов в числовые значения
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    features, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)

print(f"Обучающих примеров: {X_train.shape[0]}")
print(f"Тестовых примеров: {X_test.shape[0]}")

# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Вычисление весов классов
unique_classes = np.unique(y_train)  # Получаем уникальные классы из y_train
class_weights = {
    cls: len(y_train) / (len(unique_classes) * np.sum(y_train == cls))
    for cls in unique_classes
}

# Обучение модели с весами классов
model = RandomForestClassifier(class_weight=class_weights, random_state=42)
model.fit(X_train_scaled, y_train)

# Оценка модели
y_pred = model.predict(X_test_scaled)
print("Отчет по классификации:")
print(classification_report(
    y_test,
    y_pred,
    target_names=label_encoder.classes_,
    labels=np.arange(len(label_encoder.classes_)),
    zero_division=0
))

# Сохранение модели и scaler
model_path = "random_forest_model.pkl"
scaler_path = "scaler.pkl"
label_encoder_path = "label_encoder.pkl"

# Сохраняем модель
with open(model_path, "wb") as f:
    pickle.dump(model, f)
print(f"Модель сохранена в {model_path}")

# Сохраняем scaler
with open(scaler_path, "wb") as f:
    pickle.dump(scaler, f)
print(f"Scaler сохранен в {scaler_path}")

# Сохраняем label_encoder
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)
print(f"LabelEncoder сохранен в {label_encoder_path}")

Обучающих примеров: 6518
Тестовых примеров: 1630
Отчет по классификации:
                         precision    recall  f1-score   support

            heiko-dachi       0.35      0.63      0.45       100
   l-chudan-gyaku-tsuki       0.17      0.06      0.09        16
l-chudan-gyaku-uchi-uke       0.12      0.14      0.13        36
      l-chudan-mae-geri       0.43      0.40      0.41        15
      l-chudan-osae-uke       0.35      0.24      0.28        34
     l-chudan-shuto-uke       0.21      0.23      0.22        80
       l-chudan-tettsui       0.36      0.25      0.29        20
  l-chudan-yoko-tettsui       0.25      0.11      0.15        19
          l-gedan-barai       0.17      0.06      0.09        17
        l-jodan-age-uke       0.24      0.24      0.24        34
     l-jodan-haiwan-uke       0.31      0.24      0.27        21
  l-jodan-shuto-age-uke       0.45      0.25      0.32        20
        l-kokutsu-dachi       0.16      0.30      0.21       107
              l-